In [ ]:
import pandas as pd
fnl=pd.read_csv('/kaggle/input/standard-scaler-final-out/final.csv',index_col=0)

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        df[col]=df[col].fillna(-1)
        df[col]=df[col].replace([np.inf,-np.inf],-1)
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
fnl=reduce_mem_usage(fnl)

In [ ]:
from tqdm import tqdm
import gc
pds=['day','month','week']
#cols=list(fnl)

cols = ["TransactionAmt", "dist1", "dist2"] + ["C" + str(i) for i in range(1, 15)] + \
            ["D" + str(i) for i in range(1, 16)] 
for pd in pds:
  for col in tqdm(cols):
    col=str(col)
    use_col=str(col)+'_'+pd+'_ss'
#    if use_col in cols:
    fnl['a']=fnl.groupby([pd])[col].transform('max')
    fnl['b']=fnl.groupby([pd])[col].transform('min')
    fnl[use_col]=fnl[col]-fnl['b']/(fnl['a']-fnl['b'])
    use_col=str(col)+'_'+pd+'_norm'
#    if use_col in cols:
    fnl['a']=fnl.groupby([pd])[col].transform('std')
    fnl['b']=fnl.groupby([pd])[col].transform('mean')
    fnl[use_col]=(fnl[col]-fnl['a'])/fnl['b']
  
  try:
      fnl=reduce_mem_usage(fnl)
  except:
    continue
del([fnl['a'],fnl['b']])
gc.collect()

In [ ]:
ls=[]
for col in cols:
    ls+=list(fnl.filter(regex=col))
len(ls)

In [ ]:
fnl=fnl.replace([np.inf,-np.inf],0)
import pandas as pd
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
trn=pd.DataFrame(ss.fit_transform(fnl[ls]))
trn.columns=ls
fnl=fnl.drop([ls],1)
fnl=pd.concat([fnl,trn],1)
fnl=fnl.loc[:,~(fnl.columns.duplicated())]
fnl=reduce_mem_usage(fnl)
fnl=fnl.select_dtypes(include=['float16','int8'])
fnl.to_csv('prepared.csv')